# Gaussian process regression: Introduction

Let's generate some fake, one dimensional data

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

ndim = 10

x = np.arange(ndim)[:, np.newaxis]
y = np.arange(ndim)[:, np.newaxis]

In [ ]:
np.random.seed(42)
y = np.random.rand(ndim)[:, None]
y -= y.mean()
x = np.arange(len(y))[:, None]
yerr = y.std() / 2 * np.ones_like(x)

plt.errorbar(x, y, yerr, fmt='.', color='k')
plt.xlabel('x')
plt.ylabel('y');

# Linear regression

Solutions to the [ordinary least squares](https://en.wikipedia.org/wiki/Ordinary_least_squares) estimators $\hat{\beta}$ are

$$ \hat{\beta} = ({\bf X}^\top {\bf N}^{-1} {\bf X})^{-1} {\bf X}^\top {\bf N}^{-1} y,$$

and their uncertainties are given by

$$ \mathrm{cov} = {\bf X}^\top {\bf N}^{-1} {\bf X}, $$

where ${\bf N}$ is the matrix of uncertainties on measurements $y$ (${\bf N}$ is diagonal for measurements with independent Gaussian uncertainties, in this case). 

In [ ]:
# Append a column of ones next to the `x` values using `np.vander`: 
X = np.vander(x.ravel(), 2)
inv_N = np.linalg.inv(np.identity(len(x)) * yerr**2)

# Solve linear regression: 
betas = np.linalg.inv(X.T @ inv_N @ X) @ X.T @ inv_N @ y
cov = np.linalg.inv(X.T @ inv_N @ X)

# Compute best fit line: 
best_fit = X @ betas 
err = np.sqrt(np.diag(cov))

Plot the result and its uncertainty (here we're plotting just the uncertainty in the intercept and ignoring the uncertainty in the slope, for this example)

In [ ]:
plt.errorbar(x, y, yerr, fmt='.', color='k')
plt.plot(x, best_fit)
plt.fill_between(x.ravel(), best_fit.ravel()-err[1], best_fit.ravel()+err[1], alpha=0.3)
plt.xlabel('x')
plt.ylabel('y');

# Gaussian process regression

From [Rasmussen and Williams (2006)](http://www.gaussianprocess.org/gpml/), the predicted mean is going to be (Equation 2.25) 

$$ \mu = {\bf k_*}^\top ({\bf K} + \sigma_n {\bf I})^{-1} {\bf y},  $$

and the covariance is going to be (Equation 2.26)

$$ \mathrm{cov} = k({\bf x_*}, {\bf x_*})- {\bf k_*}^\top ({\bf K }+ \sigma_n {\bf I})^{-1} {\bf k_*}, $$ 

and therefore the error on the predictions will be

$$ \mathrm{err} = \mathrm{diag}\left( \sqrt{\mathrm{cov}} \right).$$

Define a "squared exponential" (Gaussian) kernel function

$$ K({\bf x_1}, {\bf x_2}) = \sigma^2 \exp\left(-\frac{(x_1 - x_2)^2)}{2 \ell^2}\right)   $$

In [ ]:
def square_distance(x1, x2): 
    """
    Compute the squared distance between two vectors `x1` and `x2` 
    """
    return np.sum(x1**2, axis=1)[:, np.newaxis] + np.sum(x2**2, axis=1) - 2 * np.dot(x1, x2.T)

def sq_exp_kernel(x1, x2, sigma=1, ell=1): 
    """
    Gaussian Kernel function
    """
    sqdist = square_distance(x1, x2)

    return sigma**2 * np.exp(-0.5 * sqdist / ell**2)

def gaussian_process_regression(x, y, yerr, xtest, kernel): 
    """
    Gaussian process regression for column vectors `x` and `y` with 
    uncertainties `yerr`; predict values at `xtest` using `kernel`
    """
    K = kernel(x, x) + np.identity(len(x)) * yerr**2
    k_s = kernel(x, xtest)
    k_ss = kernel(xtest, xtest)
    inv_K = np.linalg.inv(K)
    mu = k_s.T @ inv_K @ y
    cov = k_ss - k_s.T @ inv_K @ k_s
    return mu, cov

In [ ]:
N = 100
xtest = np.linspace(x.min(), x.max(), N)[:, np.newaxis]

mu, cov = gaussian_process_regression(x, y, yerr, xtest, sq_exp_kernel)

err = np.sqrt(np.diagonal(cov))

plt.errorbar(x, y, yerr, fmt='.', color='k')
plt.plot(xtest.ravel(), mu.ravel(), label='GP mean')
plt.fill_between(xtest.ravel(), mu.ravel()-err, mu.ravel()+err, alpha=0.3, label='GP uncertainty')
plt.legend(loc='upper center')
plt.xlabel('x')
plt.ylabel('y');

### Make an interactive plot

which allows you to vary the $\ell$ hyperparameter and see the results: 

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def gp_interact(ell):
    def sq_exp_kernel_interactive(x1, x2, sigma=1, ell=ell): 
        """
        Interactive Gaussian Kernel function
        """
        sqdist = np.sum(x1**2, axis=1)[:, np.newaxis] + np.sum(x2**2, axis=1) - 2 * np.dot(x1, x2.T)

        return sigma**2 * np.exp(-0.5 * sqdist / ell**2)
    
    mu, cov = gaussian_process_regression(x, y, yerr, xtest, sq_exp_kernel_interactive)
    err = np.sqrt(np.diag(cov))

    plt.errorbar(x, y, yerr, fmt='.', color='k')
    plt.plot(xtest.ravel(), mu.ravel())
    plt.fill_between(xtest.ravel(),  mu.ravel()-err, mu.ravel()+err, alpha=0.2)
    plt.gca().set(xlabel='x', ylabel='y')
    plt.show()

print('Vary the hyperparameter "ell", which defines the autocorrelation timescale:')
interactive_plot = interactive(gp_interact, ell=(1, 10, 1))
output = interactive_plot.children[-1]
interactive_plot